# Worksheet 08

Name: Mark Yang

UID: U27016194

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [171]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

np.random.seed(42)
random.seed(42)

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[5.496714153011233, 4.861735698828816, 5.6476885381006925, 6.523029856408026, 4.765846625276664, 4.765863043050819, 6.5792128155073915, 5.7674347291529084, 4.530525614065048, 5.542560043585965]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [172]:
s2 = np.random.normal( 8 , 1 , 10 ).tolist()
print(s2)

[7.536582307187538, 7.5342702464297435, 8.241962271566035, 6.086719755342202, 6.275082167486968, 7.437712470759028, 6.987168879665576, 8.314247332595274, 7.091975924478789, 6.587696298664708]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `s1` and remove it from `s1`, if T then pick the last data point from `s2` and remove it from `s2`. Add these 10 points to a list called `data`.

In [173]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1[i]
        data.append(p1)
    else:
        p2 = s2[i]
        data.append(p2)
print(data)

[5.496714153011233, 4.861735698828816, 8.241962271566035, 6.523029856408026, 4.765846625276664, 4.765863043050819, 6.5792128155073915, 5.7674347291529084, 7.091975924478789, 5.542560043585965]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

The mean, variance, and probability of each mixed distribution is

$\mu_1 = 5, \sigma_1 = 1, \pi_1 = 0.5$

$\mu_2 = 8, \sigma_2 = 1, \pi_2 = 0.5$

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [174]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) ,  sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


[5.496714153011233, 4.861735698828816, 4.765846625276664, 4.765863043050819, 5.7674347291529084, 5.542560043585965]
[8.241962271566035, 6.523029856408026, 6.5792128155073915, 7.091975924478789]
P(S_1) = 0.6,  P(S_2) = 0.4
mean_1 = 5.200025715484401,  mean_2 = 7.10904521699006
var_1 = 0.16979260459324919,  var_2 = 0.47698219744873016


No, in general the values are not accurate. The mean are fairly close, with $5 \approx 5.20$ and $8 \approx 7.11$. However, the variances are way off, $\sigma_1 = 0.170, \sigma_2 = 0.477$, under one fifth and one half their true value

e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [175]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append(pdf_i[0] * prob_s[0] / prob_x)
    prob_s1_x.append(pdf_i[1] * prob_s[1] / prob_x)

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  5.496714153011233
probability of observing that point if it came from cluster 0 =  0.5104867121434126
probability of observing that point if it came from cluster 1 =  0.0027620215227537937
point =  4.861735698828816
probability of observing that point if it came from cluster 0 =  0.32286083627057477
probability of observing that point if it came from cluster 1 =  1.2649959667613473e-05
point =  8.241962271566035
probability of observing that point if it came from cluster 0 =  4.715316310041886e-70
probability of observing that point if it came from cluster 1 =  0.049817205435799435
point =  6.523029856408026
probability of observing that point if it came from cluster 0 =  1.5389855095364956e-13
probability of observing that point if it came from cluster 1 =  0.39322291737111464
point =  4.765846625276664
probability of observing that point if it came from cluster 0 =  0.08935146518470068
probability of observing that point if it came from cluster 1 =  4.807982215661206e-06
poi

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [176]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x),
        sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]
var = [sum([p*(x - mean[0])**2 for x, p in zip(data, prob_s0_x)]) / sum(prob_s0_x),
       sum([p*(x - mean[1])**2 for x, p in zip(data, prob_s1_x)]) / sum(prob_s1_x) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.6,  P(S_2) = 0.4
mean_1 = 5.142246302672242,  mean_2 = 6.94386158379264
var_1 = 0.15128153484837978,  var_2 = 0.6138336430664507


Comparing to the original values:

$\mu_1 = 5.20,  \mu_2 = 7.11, \sigma_1 = 0.180,  \sigma_2 = 0.477$

In this case, we see that the results aren't significantly more accurate. $\sigma_1, \sigma_2$ improved, while the $\mu_1, \mu_2$ are further from their true value. The $P(S_j)$ are the same.

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [177]:
from scipy.stats import norm

new_prob_s0_x = [] # P(S_0 | X_i)
new_prob_s1_x = [] # P(S_1 | X_i)

k = 2

for p in data:
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    new_prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    new_prob_s0_x.append(pdf_i[0] * prob_s[0] / new_prob_x)
    new_prob_s1_x.append(pdf_i[1] * prob_s[1] / new_prob_x)

probs = zip(data, new_prob_s0_x, new_prob_s1_x)
for x, y0, yhat0, y1, yhat1 in zip(data, prob_s0_x, new_prob_s0_x, prob_s1_x, new_prob_s1_x):
    print("Point:", x)
    print("Group 0 old probability:", y0, "Recalculated probability:", yhat0)
    print("Group 1 old probability:", y1, "Recalculated probability:", yhat1)
    print()


Point: 5.496714153011233
Group 0 old probability: 0.9964059206859294 Recalculated probability: 0.8629513602209493
Group 1 old probability: 0.0035940793140706246 Recalculated probability: 0.13704863977905068

Point: 4.861735698828816
Group 0 old probability: 0.9999738801204088 Recalculated probability: 0.9970990387097846
Group 1 old probability: 2.6119879591278043e-05 Recalculated probability: 0.0029009612902154004

Point: 8.241962271566035
Group 0 old probability: 1.4197854743534202e-68 Recalculated probability: 3.8981633643430788e-90
Group 1 old probability: 1.0 Recalculated probability: 1.0

Point: 6.523029856408026
Group 0 old probability: 5.870660539667122e-13 Recalculated probability: 6.260699121979705e-18
Group 1 old probability: 0.9999999999994129 Recalculated probability: 1.0

Point: 4.765846625276664
Group 0 old probability: 0.9999641281036689 Recalculated probability: 0.9933450151150846
Group 1 old probability: 3.587189633121486e-05 Recalculated probability: 0.006654984884915

We see that the recalculated probabilities are more certain about some points, like those greater than 5.7. Other points it is less certain about, especially the 5.54 point where it was certain it was 99% 0 before but now is only 71% certain.

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [178]:
new_assignment = [np.argmax(x) for x in zip(new_prob_s0_x,new_prob_s1_x)]
print("New assignment for each point:")
print(new_assignment)

New assignment for each point:
[0, 0, 1, 1, 0, 0, 1, 1, 1, 0]
